### Searching Unstructured and Structured Data: Assignment 1 ### 

The cell below contains the necessary code we were provided with.

In [6]:
import collections
import io
import logging
import sys
import time
import pyndri
import os
import re
from subprocess import PIPE, run
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

index = pyndri.Index('index/')
token2id, id2token, _ = index.get_dictionary()

def write_run(model_name, data, out_f, max_objects_per_query=sys.maxsize, skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.', subject_id)
            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))

def parse_topics(file_or_files, max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None
    topics = collections.OrderedDict()
    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]
    for f in file_or_files:
        assert isinstance(f, io.IOBase)
        for line in f:
            assert(isinstance(line, str))
            line = line.strip()
            if not line:
                continue
            topic_id, terms = line.split(delimiter, 1)
            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).', topic_id, topics[topic_id], terms)
            topics[topic_id] = terms
            if max_topics > 0 and len(topics) >= max_topics:
                break
    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

num_documents = index.maximum_document() - index.document_base()
dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

start_time = time.time()
document_lengths = {}
unique_terms_per_document = {}
inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)
total_terms = 0
total_nr_docs = index.maximum_document() - index.document_base()
ext_doc_ids = {}
ext_to_int_doc_ids = {}

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)
    ext_doc_ids[int_doc_id] = ext_doc_id
    ext_to_int_doc_ids[ext_doc_id] = int_doc_id
    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())
    document_lengths[int_doc_id] = document_length
    total_terms += document_length
    unique_terms_per_document[int_doc_id] = len(document_bow)
    for query_term_id in query_term_ids:
        assert query_term_id is not None
        document_term_frequency = document_bow.get(query_term_id, 0)
        if document_term_frequency == 0:
            continue
        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency

avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 51.61398696899414 seconds.


### Task 1: Implement and compare lexical IR methods [40 points] ### 

Specify the directory to trec eval in the first cell. The code in the second cell creates all necessary functions and the third cell contains the code for generating the results.

In [49]:
trec_eval_directory = '../trec_eval/'

In [116]:
def run_retrieval(model_name, score_fn):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = 'results_task1/{}.run'.format(model_name)
    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    data = {}

    # TODO: fill the data dictionary. 
    # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
    
    for query_id in tokenized_queries:
        query = tokenized_queries[query_id]
        doc_ids = set([doc_id for term in query for doc_id in inverted_index[term].keys()])
        data[query_id] = []
        for doc_id in doc_ids:
            score = 0
            for term_id in query:
                doc_term_freq = len(inverted_index.get(term_id, 0))
                score += score_fn(doc_id, term_id, doc_term_freq)
            data[query_id].append((score, ext_doc_ids[doc_id]))
    
    print('Retrieval took', time.time() - retrieval_start_time, 'seconds.')
    
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)


def get_tf(query_term_id, int_document_id=None):
    """
    Returns term frequency for a document if a document_id is given,
    or for the entire collection if no document_id is given.
    """
    if int_document_id:
        return float(inverted_index.get(query_term_id, 0).get(int_document_id, 0))
    else:
        return collection_frequencies.get(query_term_id, 0)

def get_bg_prob(query_term_id, document_term_freq=None):
    tf_col = get_tf(query_term_id)
    bg_prob = tf_col/total_terms
    return bg_prob
    
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """

    # TODO implement the function
    idf = np.log(total_nr_docs/document_term_freq)
    tf = get_tf(query_term_id, int_document_id)
    score = np.log(1 + tf) * idf
    return score

def bm25(int_document_id, query_term_id, document_term_freq, k1=1.2, b=0.75):
    tf = get_tf(query_term_id, int_document_id)
    doc_len = document_lengths[int_document_id]
    term_score = ((k1+1) * tf)/(k1*((1-b)+b*(doc_len/avg_doc_length))+tf)
    idf = np.log(total_nr_docs/document_term_freq)
    score = term_score * idf
    return score

def jelinek_mercer(int_document_id, query_term_id, document_term_freq, l=0.1):
    tf_doc = get_tf(query_term_id, int_document_id)
    tf_col = get_tf(query_term_id)
    doc_len = document_lengths[int_document_id]
    probability = l*(tf_doc/doc_len)+(1-l)*(tf_col/total_terms)
    return np.log(probability)

def dirichlet_prior(int_document_id, query_term_id, document_term_freq, mu=1000):
    tf = get_tf(query_term_id, int_document_id)
    bg_prob = get_bg_prob(query_term_id, document_term_freq)
    doc_len = document_lengths[int_document_id]
    probability = (tf + mu*bg_prob)/(doc_len + mu)
    return np.log(probability)

def absolute_discounting(int_document_id, query_term_id, document_term_freq, delta=0.9):
    bg_prob = get_bg_prob(query_term_id, document_term_freq)
    tf = get_tf(query_term_id, int_document_id)
    doc_len = document_lengths[int_document_id]
    unique_terms = unique_terms_per_document[int_document_id]
    probability = (max(tf-delta, 0)/doc_len)+(delta*unique_terms/doc_len)*bg_prob
    return np.log(probability)

def get_hyper_parameter_values():
    filenames = ['jelinek_mercer', 'dirichlet_prior', 'absolute_discounting']
    measure = 'ndcg_cut_10'
    best_parameters = {}
    data = []
    names = []
    print('\nMeasure:', measure)
    for lm in filenames:
        files = [f for f in os.listdir('results_task1') if re.match(lm, f) and f != '.DS_Store']
        for file in files:
            result = run('./' + trec_eval_directory + 'trec_eval -m all_trec ap_88_89/qrel_validation results_task1/' + file + ' | grep -E "^' + measure + '\s"', shell=True, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            score = result.stdout.split('\t')[2].strip()
            param = re.search(re.compile('\d.\d*'), file).group()
            print(lm, param, score)
            data.append(score)
            names.append(lm + '_' + param)
            if lm not in best_parameters:
                best_parameters[lm] = {'best_param': param, 'score': score}
                continue
            if score > best_parameters[lm]['score']:
                best_parameters[lm]['best_param'] = param
                best_parameters[lm]['score'] = score

    width = 1/1.5
    plt.bar(range(len(data)), data, 1/1.5, align='edge', color='gray')
    axes = plt.gca()
    axes.set_ylim([0,0.5])
    plt.xticks(range(len(data)), names, rotation=60)
    plt.ylabel('NDCG@10')
    plt.tight_layout()
    plt.savefig('plots/' + measure + '.png')
    plt.title('Scores of ' + measure)
    plt.close() 

    print(best_parameters)

def get_trec_eval_results(measures, output_dir, directory=None):
    """
    Prints TREC Eval results and writes results to file.
    """
    if directory:
        files = sorted([f for f in os.listdir(directory) if f != '.DS_Store'])
    else:
        directory = 'results_task1/'
        files = ['tfidf.run', 'bm25.run', 'jelinek_mercer0.1.run', 'dirichlet_prior1000.run', 'absolute_discounting0.9.run']
    
    for measure in measures:
        results = []
        firstline = True
        output = open(output_dir + measure + '.txt', 'w')
        print('\nMeasure:', measure)
        for file in files:
            result = run('./' + trec_eval_directory + 'trec_eval -m all_trec ap_88_89/qrel_test ' + directory + file + ' -q | grep -E "^' + measure + '\s"', shell=True, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            scores = [float(line.split('\t')[2]) for line in result.stdout.split('\n')[:-2]] #[:-2] because last element is empty and second last element is score for all
            score = round(sum(scores)/len(scores),3)
            if firstline:
                queries = [line.split('\t')[1] for line in result.stdout.split('\n')[:-2]]
                output.write(measure + '\t' + '\t'.join(queries) + '\n')
                firstline = False
            print(file, score)
            output.write(file + '\t' + '\t'.join(str(x) for x in scores) + '\n')
        output.close() 

def test_methods(file):
    """
    Performs two-tailed Student t-test between MAP results.
    """
    lines = [[line.split('\t')[0],[float(score) for score in line.split('\t')[1:]]] for line in open(file).readlines()]
    count = 0
    results = []
    for i in range(1, len(lines)):
        for e in range(i+1, len(lines)):
            count += 1
            t = stats.ttest_rel(lines[i][1], lines[e][1])
            print(lines[i][0], lines[e][0], t.pvalue)
    print('Number of comparisons:', count)

In [117]:
# combining the two functions above: 
# run_retrieval('tfidf', tfidf)
# run_retrieval('bm25', bm25)
# run_retrieval('jelinek_mercer0.1', jelinek_mercer)
# run_retrieval('dirichlet_prior1000', dirichlet_prior)
# run_retrieval('absolute_discounting0.9', absolute_discounting)

# TODO implement tools to help you with the analysis of the results.
print('Best hyper parameter values')
get_hyper_parameter_values()
print('\nGetting TREC Eval results on test set')
get_trec_eval_results(["ndcg_cut_10", "map_cut_1000", "P_5", "recall_1000"], 'trec_eval_results_task1/')
print('\nSignificance testing')
test_methods('trec_eval_results_task1/map_cut_1000.txt')

Best hyper parameter values

Measure: ndcg_cut_10
jelinek_mercer 0.9 0.3676
jelinek_mercer 0.5 0.3823
jelinek_mercer 0.1 0.3991
dirichlet_prior 1000 0.4002
dirichlet_prior 500 0.4055
dirichlet_prior 1500 0.4026
absolute_discounting 0.1 0.3614
absolute_discounting 0.5 0.3768
absolute_discounting 0.9 0.3950
{'jelinek_mercer': {'best_param': '0.1', 'score': '0.3991'}, 'dirichlet_prior': {'best_param': '500', 'score': '0.4055'}, 'absolute_discounting': {'best_param': '0.9', 'score': '0.3950'}}

Getting TREC Eval results on test set

Measure: ndcg_cut_10
tfidf.run 0.417
bm25.run 0.409
jelinek_mercer0.1.run 0.349
dirichlet_prior1000.run 0.414
absolute_discounting0.9.run 0.386

Measure: map_cut_1000
tfidf.run 0.216
bm25.run 0.217
jelinek_mercer0.1.run 0.189
dirichlet_prior1000.run 0.213
absolute_discounting0.9.run 0.203

Measure: P_5
tfidf.run 0.432
bm25.run 0.413
jelinek_mercer0.1.run 0.345
dirichlet_prior1000.run 0.413
absolute_discounting0.9.run 0.398

Measure: recall_1000
tfidf.run 0.651


### Task 2: Latent Semantic Models (LSMs) [20 points] ###

The code below generates our results for the second task. 

In [101]:
import copy
import gensim
import logging
import pyndri
import pyndri.compat
from gensim import corpora, models, similarities
import collections
from scipy import stats

dictionary = pyndri.extract_dictionary(index)
corpus = IndriCorpus(index, dictionary)

document_candidates = {}

for line in open('results_task1/tfidf.run'):
    line = line.split(' ')
    query = int(line[0])
    if query not in document_candidates:
        document_candidates[query] = []
    document_candidates[query].append(ext_to_int_doc_ids[line[2]])

class IndriCorpus(gensim.interfaces.CorpusABC):

    def __init__(self, index, dictionary, max_documents=None):
        assert isinstance(index, pyndri.Index)

        self.index = index
        self.dictionary = dictionary

        self.max_documents = max_documents

    def _maximum_document(self):
        if self.max_documents is None:
            return self.index.maximum_document()
        else:
            return min(
                self.max_documents + self.index.document_base(),
                self.index.maximum_document())

    def __iter__(self):
        for int_doc_id in range(self.index.document_base(),
                                self._maximum_document()):
            ext_doc_id, tokens = self.index.document(int_doc_id)

            # Compared to IndriSentences, the only difference is the
            # switching of tuple(self.dictionary[token_id] ...) by
            # sorted(collections.Counter(token_id ...).items()).
            yield sorted(collections.Counter(
                token_id
                for token_id in tokens
                if token_id > 0 and token_id in self.dictionary).items())

def generate_lsm_results(run_out_path, model_name, k):
    
    start_time = time.time()
    
    print('creating', model_name, 'model')

    if model_name == 'lda':
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=k, id2word=dictionary, minimum_probability=0.0)
    else:
        model = gensim.models.lsimodel.LsiModel(corpus, num_topics=k, id2word=dictionary)
    
    print('calculating query document similarity')
    
    data = {}
    
    for query_id in queries:
        data[query_id] = []
        query_bow = dictionary.doc2bow(queries[query_id].lower().split())
        query_vec = model[query_bow]
        query = tokenized_queries[query_id]
        docs = document_candidates[int(query_id)]
        for doc_id in docs:
            doc_token_ids = index.document(doc_id)[1]
            doc_bow = dictionary.doc2bow(doc_token_ids)
            doc_vec = model[doc_bow]
            if model_name == 'lda':
                score = 0 - stats.entropy(query_vec, doc_vec)[1]
            else:
                score = gensim.matutils.cossim(query_vec, doc_vec)
            data[query_id].append((score, ext_doc_ids[doc_id]))
            
    print('LSM took', time.time() - start_time, 'seconds')
    
    with open(run_out_path, 'w') as f_out:
        write_run(
        model_name='LDA'+str(k),
        data=data,
        out_f=f_out,
        max_objects_per_query=1000)

#generate lsi and lda models with 2 topics
# generate_lsm_results('results_task2/LSI_2.run', 'lsi', 2)
# generate_lsm_results('results_task2/LDA_2.run', 'lda', 2)

get_trec_eval_results(["ndcg_cut_10", "map_cut_1000"], 'trec_eval_results_task2/', directory='results_task2/')


Measure: ndcg_cut_10











Measure: map_cut_1000












### Task 3:  Word embeddings for ranking [10 points] ###

The code below generates our results for the third task.

### Task 4: Learning to rank (LTR) [10 points] ###